# Style Transfer with Deep Neural Networks
In this notebook, we’ll recreate a style transfer method that is outlined in the paper, Image Style Transfer Using Convolutional Neural Networks, by Gatys in PyTorch.

In [ ]:
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt 
import numpy as np

import torch
import torch.optim as optim
import requests
from torchvision import transforms, models




## Loading the vgg 19 network from the Py-torch library

###### vgg19.features  are convolutional  and pooling layers
###### vgg19.classifier  are convolutional  and pooling layers

In [ ]:
vgg = models.vgg19(pretrained = True).features


for param in vgg.parameters():
    param.requires_grad_(False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vgg.to(device)

# Load in Content and Style Images

In [ ]:
def load_image(img_path , max_size = 400 , shape= None):
    
    if "https" in img_path:
        
        respose = requests.get(img_path)
        image   = Image.open()
        
    else:
        image   = Image.open(img_path).convert('RGB')
        
    ## decrease the size of the image
    if max(image.size) > max_size:
                    size = max_size 
    else :
        size = max(image.size)
        
    if shape is not None:
        size = shape
        
    transformer = transforms.Compose([ transforms.Resize(size),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406),
                                                           (0.229, 0.224, 0.225))])
    
    image = transformer(image)[:3 , : , :].unsqueeze(0)
    
    return image


# the content  image

In [ ]:
content = load_image('images/octopus.jpg').to(device)

style   = load_image('images/hockney.jpg' , shape=content.shape[-2:]).to(device)



# un normalizing the images

In [ ]:
def im_convert(tensor):
    
    image = tensor.to("cpu").clone().detach()
    
    image = image.numpy().squeeze()
    
    image = image.transpose(1 , 2, 0)
    
    image = image*np.array((0.229, 0.224, 0.225))+ np.array((0.485, 0.456, 0.406))
        
    image = image.clip(0,1)
    
    
    return image

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
# content and style ims side-by-side
ax1.imshow(im_convert(content))
ax2.imshow(im_convert(style))

Content and Style Features
Below, complete the mapping of layer names to the names found in the paper for the content representation and the style representation.

In [ ]:
def get_features(image, model, layers = None):
    
    if layers is None:
        layers = { '0' : 'conv1_1',
                   '5' : 'conv2_1',
                   '10': 'conv3_1',
                   '19': 'conv4_1',
                   '21': 'conv4_2',
                   '28': 'conv5_1',}
        
    features ={}
    x = image
    
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
            
    return features

#  calculate the Gram Matrix

Get the depth, height, and width of a tensor using batch_size, d, h, w = tensor.size
Reshape that tensor so that the spatial dimensions are flattened
Calculate the gram matrix by multiplying the reshaped tensor by it's transpose

In [ ]:
def gram_matrix(tensor):
    
    _, depth , height, width = tensor.size()
    
    tensor = tensor.view(depth, height*width)
    
    
    
    #calculating  the gram matrix
    gram = torch.mm(tensor, tensor.t())
    
    
    return gram

## Putting it all together 

In [ ]:
content_features  = get_features(content, vgg)
style_features    = get_features(style, vgg) 



# Loss calculation:
style_grams = {layer : gram_matrix(style_features[layer]) for layer in style_features }
target = content.clone().requires_grad_(True).to(device)

# Loss and Weights


Content and Style Weight
Just like in the paper, we define an alpha (content_weight) and a beta (style_weight). This ratio will affect how stylized your final image is. It's recommended that you leave the content_weight = 1 and set the style_weight to achieve the ratio you want.

In [ ]:
style_weights  ={'conv1_1' : 1.,
                 'conv2_1'  : 0.75,
                 'conv2_1'  : 0.2,
                 'conv2_1'  : 0.2,
                 'conv2_1'  : 0.2}


content_weight = 1  # alpha
style_weight = 1e6  # beta

Content Loss
The content loss will be the mean squared difference between the target and content features at layer conv4_2. This can be calculated as follows:

content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2'])**2)
Style Loss
The style loss is calculated in a similar way, only you have to iterate through a number of layers, specified by name in our dictionary style_weights.

You'll calculate the gram matrix for the target image, target_gram and style image style_gram at each of these layers and compare those gram matrices, calculating the layer_style_loss. Later, you'll see that this value is normalized by the size of the layer.

Total Loss
Finally, you'll create the total loss by adding up the style and content losses and weighting them with your specified alpha and beta!

Intermittently, we'll print out this loss; don't be alarmed if the loss is very large. It takes some time for an image's style to change and you should focus on the appearance of your target image rather than any loss value. Still, you should see that this loss decreases over some number of iterations.

In [ ]:
show_every = 400

optimizer = optim.Adam([target], lr = 0.003)

steps     = 2000
for ii in range(1 , steps +1):
    
    
    target_features = get_features(target, vgg)
    
    
    content_loss    = torch.mean((target_features['conv4_2'] -
                                  content_features['conv4_2'])**2)
    style_loss = 0 
    
    
    for layer in style_weights:
        target_feature = target_features[layer]
        target_gram = gram_matrix(target_feature)
        _, d, h, w = target_feature.shape
        style_gram = style_grams[layer]
        # the style loss for one layer, weighted appropriately
        layer_style_loss = style_weights[layer] * \
                            torch.mean((target_gram - style_gram)**2)
        # add to the style loss
        style_loss += layer_style_loss / (d * h * w)
        
        #total loss 
    total_loss   = content_weight * \
                   content_loss +   \
                   style_weight *   \
                   style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    
    
    if ii % show_every ==0:
        print('total loss ' , total_loss.items())
        plt.imshow(in_convert(target))
        plt.show()